In [18]:
import numpy as np

gate = {
    "I": np.array([[1, 0], [0, 1]], dtype=np.complex128),
    "RX": lambda theta: np.array(
        [
            [np.cos(theta / 2), -1j * np.sin(theta / 2)],
            [-1j * np.sin(theta / 2), np.cos(theta / 2)],
        ],
        dtype=np.complex128,
    ),
    "RY": lambda theta: np.array(
        [
            [np.cos(theta / 2), -np.sin(theta / 2)],
            [np.sin(theta / 2), np.cos(theta / 2)],
        ],
        dtype=np.complex128,
    ),
    "RZ": lambda theta: np.array(
        [[np.exp(-1j * theta / 2), 0], [0, np.exp(1j * theta / 2)]], dtype=np.complex128
    ),
    "H": np.array([[1, 1], [1, -1]], dtype=np.complex128) / np.sqrt(2)
}

import qiskit

num_qubits = 3
qc = qiskit.QuantumCircuit(num_qubits)
qc.rz(0.3, 0)
qc.cx(0, 1)
qc.h(2)
qc.h(0)
qc.rx(0.5, 1)
qc.h(2)
qc.cx(0, 1)
from qimax import converter, circuit, splitter
num_qubits = qc.num_qubits
matrices = []
Us: list[circuit.U] = []
Usm = [] # [U_{0:m-1}, U_{1:m-1}, ... U_{m-1:m-1}]
index = 0
qasm_gates = converter.qasm_to_qasmgates(converter.qc_to_qasm(qc))
qcs = splitter.qasmgates_to_qcs2(qasm_gates)
for qc in qcs:
    print('xxxx')
    print(qc)

rz(0.3) q[0];
cx q[0],q[1];
h q[2];
h q[0];
rx(0.5) q[1];
h q[2];
xxxx
[('RZ', 0.3, [0])]
xxxx
[('X', -999, [0, 1]), ('H', -999, [2])]
xxxx
[('H', -999, [0]), ('RX', 0.5, [1]), ('H', -999, [2])]


In [20]:
strs = []
strs.append(str(len(qcs)))
for qc in qcs:
    # Consider a U include n gates
    for gate1 in qc:
        print(gate1[0])
        if gate1[0] in ['I', 'RZ']:
            # Sparse
            if gate1[0] in ['I']:
                gate_data = [0, gate1[2][0], (gate[gate1[0]][0][0]), (gate[gate1[0]][1][1])]
            if gate1[0] in ['RZ']:
                param = gate1[1]
                gate_data = [0, gate1[2][0], gate[gate1[0]](param)[0][0], gate[gate1[0]](param)[1][1]]
        if gate1[0] in ['RX', 'RY', 'H']:
            # Dense
            if gate1[0] in ['RX', 'RY']:
                param = gate1[1]
                gate_data = [1, gate1[2][0], gate[gate1[0]](param)[0][0] + gate[gate1[0]](param)[0][1],
                         gate[gate1[0]](param)[1][0] + gate[gate1[0]](param)[1][1]]
            if gate1[0] in ['H']:
                gate_data = [1, gate1[2][0], gate[gate1[0]][0][0] + gate[gate1[0]][0][1],
                         gate[gate1[0]][1][0] + gate[gate1[0]][1][1]]
        if gate1[0] in ['X']:
            gate_data = [2, str(gate1[2][0]), (gate1[2][1])]
        for i in range(len(gate_data)):
            strs.append(gate_data[i])


with open('output.txt', 'w') as f:
    for item in strs:
        f.write("%s\n" % item)

RZ
X
H
H
RX
H


In [ ]:
lst = [1, 1]lst = [1, 1]lst = [1, 1]